In [2]:
#We import the necessary Packages
import pandas as pd
import re
import numpy as np
import datetime
import os.path
import random

DATA_DIR = os.path.abspath('/Users/attiliocastano/Documents/GitHub/amethyst-erdos-may-2021/data')
DF_Extracted=pd.read_csv(os.path.join(DATA_DIR,'Extracted_Attributes_LoanUSD_Country.csv'))

In [3]:
DF_Extracted.head()

,filename,Project_name,Project_desc,Date,Amount_loan,Currency_name,Year,Value,Amount_in_usd,Country_Code,Country,Region,Income_level
0,1990_april_24_587321468019152780_conformed-cop...,forestry sector project,Description of the Project The objectives of t...,1990-04-24,80000000.0,USD,1990,1.0,80000000.0,CIV,Cote d'Ivoire,Sub-Saharan Africa,Lower middle income
1,1990_april_24_668811468165272290_conformed-cop...,water supply project,NaN,1990-04-24,2500000.0,USD,1990,1.0,2500000.0,LCA,St. Lucia,Latin America & Caribbean,Upper middle income
2,1990_april_25_904191468298750561_conformed-cop...,environment management project,Description of the Project The objectives of t...,1990-04-25,18000000.0,USD,1990,1.0,18000000.0,POL,Poland,Europe & Central Asia,High income
3,1990_april_30_410811468040573756_conformed-cop...,rural electrification project,Description of the Project The objective of th...,1990-04-30,329000000.0,USD,1990,1.0,329000000.0,IDN,Indonesia,East Asia & Pacific,Upper middle income
4,1990_april_30_725911468042268845_conformed-cop...,third telecommunications project,Description of the Project The objectives of t...,1990-04-30,350000000.0,USD,1990,1.0,350000000.0,IDN,Indonesia,East Asia & Pacific,Upper middle income


In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [5]:
doc0 = nlp(DF_Extracted.iloc[0,2])

In [6]:
print(doc0.text)

Description of the Project The objectives of the Project are to assist the Borrower in (i) protecting the gazetted forests (ii) improving the interface between agriculture and forestry (iii) rationalizing and improving forest management and exploitation and (iv) ensuring regular wood supply and improving the efficiency of its wood processing industry The Project consists of the following parts subject to such modifications


In [7]:
doc0[0].text

'Description'

In [9]:
text_features = ['filename', 'Project_name', 'Project_desc']
DF = DF_Extracted[text_features]
DF.iloc[:,0]

for i in DF.index:
    if not(type(DF.loc[i, 'Project_desc']) == str):
        DF = DF.drop(index = i)
        
for i in DF.index:
    if not(type(DF.loc[i, 'Project_name']) == str):
        DF = DF.drop(index = i)
        
for i in DF.index:
    if not(type(DF.loc[i, 'filename']) == str):
        DF = DF.drop(index = i)
        
DF.shape

(2318, 3)

In [22]:
for token in doc0:
    print(token.text, token.pos_, token.dep_, token.lemma_)

Description NOUN ROOT description
of ADP prep of
the DET det the
Project PROPN pobj Project
The DET det the
objectives NOUN nsubj objective
of ADP prep of
the DET det the
Project PROPN pobj Project
are AUX ROOT be
to PART aux to
assist VERB xcomp assist
the DET det the
Borrower PROPN dobj Borrower
in ADP prep in
( PUNCT punct (
i NOUN pobj i
) PUNCT punct )
protecting VERB pcomp protect
the DET det the
gazetted ADJ amod gazetted
forests NOUN dobj forest
( PUNCT punct (
ii X appos ii
) PUNCT punct )
improving VERB ccomp improve
the DET det the
interface NOUN dobj interface
between ADP prep between
agriculture NOUN pobj agriculture
and CCONJ cc and
forestry NOUN conj forestry
( PUNCT punct (
iii NOUN appos iii
) PUNCT punct )
rationalizing VERB advcl rationalize
and CCONJ cc and
improving VERB conj improve
forest NOUN compound forest
management NOUN dobj management
and CCONJ cc and
exploitation NOUN conj exploitation
and CCONJ cc and
( PUNCT punct (
iv X meta iv
) PUNCT punct )
ensuring 

In [11]:
doc0.ents

(Project, Project, Borrower, Project)

In [12]:
for ent in doc0.ents:
    print(ent.label_)

ORG
ORG
PERSON
ORG


In [13]:
spacy.explain('pobj')

'object of preposition'

In [14]:
spacy.explain('dobj')

'direct object'

In [15]:
spacy.explain('compound')

'compound'

After making some exploration, we could try to extract the relevants part of the text by extrating the 'objects of preposition' (pobj) , 'direct object'(dobj)

In the example above some key words are:
Forest:...

In [16]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)
from spacy import displacy

In [17]:
for i in DF.index:
    print(DF.Project_desc.values)
    if i > 10:
        break

['Description of the Project The objectives of the Project are to assist the Borrower in (i) protecting the gazetted forests (ii) improving the interface between agriculture and forestry (iii) rationalizing and improving forest management and exploitation and (iv) ensuring regular wood supply and improving the efficiency of its wood processing industry The Project consists of the following parts subject to such modifications'
 'Description of the Project The objectives of the Project are to assist the Borrower in strengthening institutional and regulatory frameworks for environmental protection and in developing programs to improve environmental quality The Project consists of the following parts subject to such modifications thereof as the Borrower and the Bank may agree upon from time to time to achieve such objectives'
 'Description of the Project The objective of the Project is to assist the Borrower in developing and implementing an efficient rural electricity program in Indonesia

In [71]:
for i in DF.sample(10).index:    
    for token in nlp(DF.Project_name[i]):
        print(token.text, token.pos_, token.dep_)
    displacy.render(nlp(DF.Project_name[i]), jupyter = True)
    print('-----')

cotton NOUN compound
subsector NOUN compound
improvement NOUN compound
project NOUN ROOT


-----
national PROPN amod
agricultural ADJ amod
innovation NOUN compound
system NOUN compound
support NOUN compound
project NOUN nmod
proyecto X nmod
consolidaci6n NOUN nmod
del X nmod
sistema X nmod
nacional ADJ amod
de X nmod
innovacin PROPN compound
agraria PROPN nmod
del PROPN nmod
programa NOUN nsubj
nacional ADJ amod
de X relcl
innovaci6n VERB ROOT
agraria NOUN dobj


-----
ports NOUN compound
development NOUN nmod
and CCONJ cc
environmental ADJ amod
improvement NOUN conj
proj NOUN compound
ect VERB ROOT


-----
hydropower NOUN nmod
rehabilitation NOUN nmod
and CCONJ cc
system NOUN compound
control NOUN compound
project NOUN ROOT


-----
integrated VERB amod
mass NOUN compound
transit NOUN compound
systems NOUN compound
project NOUN ROOT


...

-----
tax NOUN compound
administration NOUN compound
modernization NOUN compound
project NOUN ROOT


-----
resultsbased ADJ amod
management NOUN nmod
and CCONJ cc
budgeting NOUN conj
project NOUN ROOT


-----
juntos NOUN nsubj
results NOUN dobj
for ADP prep
nutrition NOUN compound
project NOUN compound
cierre PROPN pobj
de X prep
brechas X nmod
en X prep
productos NOUN compound
priorizados NOUN advcl
dentro VERB ROOT
del PROPN amod
programa PROPN compound
articulado PROPN compound
nutricional ADJ dobj


-----
natural ADJ amod
resource NOUN compound
management NOUN compound
program NOUN ROOT


-----


In [23]:
spacy.explain('PROPN')

'proper noun'

In [40]:
for i in DF.sample(25).index:    
    doc = nlp(DF.Project_desc[i])
    displacy.render(doc, jupyter = True, style="ent")
    print('-----')

-----


-----


-----


-----


-----


...

-----


-----


-----


-----


-----


In [41]:
from spacy.matcher import DependencyMatcher

In [80]:
for i in DF.sample(25).index:
    doc = nlp(DF.Project_name[i])
    print(doc.text)
    matcher = DependencyMatcher(nlp.vocab)
    pattern = [
        {
            'RIGHT_ID': 'Root',
            'RIGHT_ATTRS': {'LOWER': 'project'}
        },
        {
            'LEFT_ID': 'Root',
            'REL_OP': '>>',
            'RIGHT_ID': 'Topic',
            'RIGHT_ATTRS': {'POS': 'NOUN'}
        }
    ]
    matcher.add('A', [pattern])
    matches = matcher(doc)
    print(matches)
    
    if len(matches)> 0:
        for match in matches:
            match_id, token_ids = match
            for i in range(len(token_ids)):
                print(pattern[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text) 
            print('+++')
            
    else:
        print('No patterns')
        
    print('--------')


    

second rural electrification project
[(14862748245026736845, [3, 2])]
Root: project
Topic: electrification
+++
--------
medical equipment project
[(14862748245026736845, [2, 1])]
Root: project
Topic: equipment
+++
--------
szczecinswinoujscie seaway and port modernization project
[(14862748245026736845, [5, 1]), (14862748245026736845, [5, 3]), (14862748245026736845, [5, 4])]
Root: project
Topic: seaway
+++
Root: project
Topic: port
+++
Root: project
Topic: modernization
+++
--------
second agricultural research management project
[(14862748245026736845, [4, 2]), (14862748245026736845, [4, 3])]
Root: project
Topic: research
+++
Root: project
Topic: management
+++
--------
generation recovery and improvement project
[(14862748245026736845, [4, 0]), (14862748245026736845, [4, 1]), (14862748245026736845, [4, 3])]
Root: project
Topic: generation
+++
Root: project
Topic: recovery
+++
Root: project
Topic: improvement
+++
--------
fuzhou nantai island periurban development project
[(1486274824

In [65]:
for i in range(0):
    print(i)